In [706]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
# from sklearn.linear_model import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score
from sklearn.preprocessing import FunctionTransformer
import sys

In [707]:
sys.path.insert(1, "../scripts")
from ml_processors import ML_Processor as MLP

mlp = MLP()

Loading data

In [708]:
chrome_mobile_data = pd.read_csv('../data/chrome_data.csv')
chrome_webview_data = pd.read_csv('../data/chrome_mobile_webView_data.csv')
facebook_data = pd.read_csv('../data/facebook_data.csv')
platform5_data = pd.read_csv('../data/platform_5.csv')
platform6_data = pd.read_csv('../data/platform_6.csv')
chrome_mobile_data.head(2)

,auction_id,experiment,date,hour,device_make,browser,yes,no,SAID_YES
0,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-04,16,Generic Smartphone,Chrome Mobile,1,0,1
1,00a1384a-5118-4d1b-925b-6cdada50318d,exposed,2020-07-06,8,Generic Smartphone,Chrome Mobile,0,1,0


### Dropping unwanted columns

In [709]:
#for mobile view
columns_to_be_dropped = ['no','SAID_YES']
chrome_mobile_data_clean = chrome_mobile_data.drop(columns_to_be_dropped,axis=1)
chrome_mobile_data_clean.head(2)

,auction_id,experiment,date,hour,device_make,browser,yes
0,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-04,16,Generic Smartphone,Chrome Mobile,1
1,00a1384a-5118-4d1b-925b-6cdada50318d,exposed,2020-07-06,8,Generic Smartphone,Chrome Mobile,0


In [710]:
chrome_webview_data.head(2)

,auction_id,experiment,date,hour,device_make,browser,yes,no,SAID_YES
0,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,Chrome Mobile WebView,0,1,0
1,01905bae-fc59-475e-b2b4-09b78163c961,exposed,2020-07-05,6,I3312,Chrome Mobile WebView,0,1,0


In [711]:
#for chrome mobile webview
columns_to_be_dropped = ['no','SAID_YES']
chrome_webview_data_clean = chrome_webview_data.drop(columns_to_be_dropped,axis=1)
chrome_webview_data_clean.head(2)

,auction_id,experiment,date,hour,device_make,browser,yes
0,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,Chrome Mobile WebView,0
1,01905bae-fc59-475e-b2b4-09b78163c961,exposed,2020-07-05,6,I3312,Chrome Mobile WebView,0


In [712]:
facebook_data.head(2)

,auction_id,experiment,date,hour,device_make,browser,yes,no,SAID_YES
0,00b6fadb-10bd-49e3-a778-290da82f7a8d,control,2020-07-08,4,Samsung SM-A202F,Facebook,1,0,1
1,023458f7-122d-475f-9525-1b9a3cc7a033,control,2020-07-03,15,MRD-LX1,Facebook,0,1,0


In [713]:
#for chrome mobile facebook
columns_to_be_dropped = ['no','SAID_YES']
facebook_data_clean = facebook_data.drop(columns_to_be_dropped,axis=1)
facebook_data_clean.head(2)

,auction_id,experiment,date,hour,device_make,browser,yes
0,00b6fadb-10bd-49e3-a778-290da82f7a8d,control,2020-07-08,4,Samsung SM-A202F,Facebook,1
1,023458f7-122d-475f-9525-1b9a3cc7a033,control,2020-07-03,15,MRD-LX1,Facebook,0


In [714]:
platform5_data.head(2)

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,23b2028c-72ed-43b3-a2d7-d772b2d4d478,exposed,2020-07-03,8,iPhone,5,Mobile Safari,0,1
1,54a62f37-0c4d-4300-a063-1c6aa99244e5,control,2020-07-03,15,iPhone,5,Mobile Safari,0,1


In [715]:
# dropping no columun for platform_5
columns_to_be_dropped = ['no']
platform5_data_clean = platform5_data.drop(columns_to_be_dropped,axis=1)
platform5_data_clean.head(2)

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes
0,23b2028c-72ed-43b3-a2d7-d772b2d4d478,exposed,2020-07-03,8,iPhone,5,Mobile Safari,0
1,54a62f37-0c4d-4300-a063-1c6aa99244e5,control,2020-07-03,15,iPhone,5,Mobile Safari,0


In [716]:
platform6_data.head(2)

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
1,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-04,16,Generic Smartphone,6,Chrome Mobile,1,0


In [717]:
#dropping no columun for paltform 6
columns_to_be_dropped = ['no']
platform6_data_clean = platform6_data.drop(columns_to_be_dropped,axis=1)
platform6_data_clean.head(2)

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes
0,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0
1,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-04,16,Generic Smartphone,6,Chrome Mobile,1


#   Splitting Numerical and catagoriacal vaiables

In [718]:
#for chrome mobile view
cat, num = mlp.sep_cat_num(chrome_mobile_data_clean)
print("Numerical Columns:", cat)
print("****************")
print("Categorical Columns:", num)

Numerical Columns: ['auction_id', 'experiment', 'date', 'device_make', 'browser']
****************
Categorical Columns: ['hour', 'yes']


In [719]:
#for chrome mobile web view
cat, num = mlp.sep_cat_num(chrome_webview_data_clean)
print("Numerical Columns:", cat)
print("****************")
print("Categorical Columns:", num)

Numerical Columns: ['auction_id', 'experiment', 'date', 'device_make', 'browser']
****************
Categorical Columns: ['hour', 'yes']


In [720]:
#for chrome facebook
cat, num = mlp.sep_cat_num(facebook_data_clean)
print("Numerical Columns:", cat)
print("****************")
print("Categorical Columns:", num)

Numerical Columns: ['auction_id', 'experiment', 'date', 'device_make', 'browser']
****************
Categorical Columns: ['hour', 'yes']


In [721]:
#for chrome platform_5
cat, num = mlp.sep_cat_num(platform5_data_clean)
print("Numerical Columns:", cat)
print("****************")
print("Categorical Columns:", num)

Numerical Columns: ['auction_id', 'experiment', 'date', 'device_make', 'browser']
****************
Categorical Columns: ['hour', 'platform_os', 'yes']


In [722]:
#for chrome platform 6 
cat, num = mlp.sep_cat_num(platform6_data_clean)
print("Numerical Columns:", cat)
print("****************")
print("Categorical Columns:", num)

Numerical Columns: ['auction_id', 'experiment', 'date', 'device_make', 'browser']
****************
Categorical Columns: ['hour', 'platform_os', 'yes']


# Splitting the columns for encoding and label encoding

In [723]:
chrome_mobile_data_clean[cat].describe()

,auction_id,experiment,date,device_make,browser
count,695,695,695,695,695
unique,695,2,8,14,1
top,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-03,Generic Smartphone,Chrome Mobile
freq,1,371,176,665,695


In [724]:
chrome_webview_data_clean[cat].describe()

,auction_id,experiment,date,device_make,browser
count,227,227,227,227,227
unique,227,2,8,79,1
top,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-03,Generic Smartphone,Chrome Mobile WebView
freq,1,180,48,43,227


In [725]:
facebook_data_clean[cat].describe()

,auction_id,experiment,date,device_make,browser
count,156,156,156,156,156
unique,156,2,8,63,1
top,00b6fadb-10bd-49e3-a778-290da82f7a8d,control,2020-07-03,Samsung SM-G960F,Facebook
freq,1,112,47,16,156


In [726]:
platform5_data_clean[cat].describe()

,auction_id,experiment,date,device_make,browser
count,17,17,17,17,17
unique,17,2,8,1,3
top,23b2028c-72ed-43b3-a2d7-d772b2d4d478,control,2020-07-03,iPhone,Mobile Safari
freq,1,13,5,17,13


In [727]:
platform6_data_clean[cat].describe()

,auction_id,experiment,date,device_make,browser
count,1226,1226,1226,1226,1226
unique,1226,2,8,115,6
top,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-03,Generic Smartphone,Chrome Mobile
freq,1,653,320,719,695


In [728]:
pipe_1 = Pipeline(
    steps=[
        ("label categories", FunctionTransformer(mlp.cat_labeler, kw_args={"cat_cols": cat})),
        ("scale data", FunctionTransformer(mlp.scaler)),
        ("separate target and features", FunctionTransformer(mlp.target_feature, kw_args={"f_r": [0, 6], "t":-1})),
        ("divide dataset", FunctionTransformer(mlp.set_splitter, kw_args={"test": 0.1, "val":0.2, "rand_state":8})),      
        ])
chrome_mobile_set = pipe_1.fit_transform(chrome_mobile_data_clean)
chrome_webview_set = pipe_1.fit_transform(chrome_webview_data_clean)
facebook_set = pipe_1.fit_transform(facebook_data_clean)
platform5_set = pipe_1.fit_transform(platform5_data_clean)
platform6_set = pipe_1.fit_transform(platform6_data_clean)

cat_labeler output...

   auction_id  experiment  date  hour  device_make  browser  yes
0           0           1     1    16            1        0    1
1           1           1     3     8            1        0    0


scaler output... 

   auction_id  experiment      date      hour  device_make  browser  yes
0    0.000000         1.0  0.142857  0.695652     0.076923      0.0  1.0
1    0.001441         1.0  0.428571  0.347826     0.076923      0.0  0.0


target_features output... 

features size: (695, 6)


set_splitter output... 

X_train shape: (512, 6)
y_train shape: (512,)
x_test shape: (70, 6)
y_test shape: (70,)
X_val shape: (113, 6)
y_val shape: (113,)
cat_labeler output...

   auction_id  experiment  date  hour  device_make  browser  yes
0           0           1     2     2            3        0    0
1           1           1     2     6           11        0    0


scaler output... 

   auction_id  experiment      date      hour  device_make  browser  yes
0    0.000000      

In [729]:
pipe_chrome_mobile =Pipeline([('scalar3',StandardScaler()),('rf_classifier',RandomForestClassifier(random_state=2))])
pipe_chrome_webview =Pipeline([('scalar3',StandardScaler()),('rf_classifier',RandomForestClassifier(random_state=2))])
pipe_facebook =Pipeline([('scalar3',StandardScaler()),('rf_classifier',RandomForestClassifier(random_state=2))])
pipe_platform5 =Pipeline([('scalar3',StandardScaler()),('rf_classifier',RandomForestClassifier(random_state=2))])
pipe_platform6 =Pipeline([('scalar3',StandardScaler()),('rf_classifier',RandomForestClassifier(random_state=2))])

In [730]:
pipe_chrome_mobile.fit(chrome_mobile_set[0], chrome_mobile_set[1])
result = pipe_chrome_mobile.score(chrome_mobile_set[4],chrome_mobile_set[5])

print(f'The accuracy we get using the chrome mobile data is: {result}')


The accuracy we get using the chrome mobile data is: 0.5398230088495575


In [731]:
pipe_chrome_webview.fit(chrome_webview_set[0], chrome_webview_set[1])
result = pipe_chrome_webview.score(chrome_webview_set[4],chrome_webview_set[5])

print(f'The accuracy we get using the chrome mobile webview data is: {result}')


The accuracy we get using the chrome mobile webview data is: 0.5405405405405406


In [732]:
pipe_facebook.fit(facebook_set[0], facebook_set[1])
result = pipe_facebook.score(facebook_set[4],facebook_set[5])

print(f'The accuracy we get using the facebook data is: {result}')


The accuracy we get using the facebook data is: 0.5


In [733]:
pipe_platform5.fit(platform5_set[0], platform5_set[1])
result = pipe_platform5.score(platform5_set[4],platform5_set[5])

print(f'The accuracy we get using the platform5 data is: {result}')


The accuracy we get using the platform5 data is: 0.3333333333333333


In [734]:
pipe_platform6.fit(platform6_set[0], platform6_set[1])
result = pipe_platform6.score(platform6_set[4],platform6_set[5])

print(f'The accuracy we get using the platform6 data is: {result}')


The accuracy we get using the platform6 data is: 0.5025125628140703


In [735]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

In [736]:

clean_data_list = [chrome_mobile_data_clean,chrome_webview_data_clean,facebook_data_clean,platform5_data_clean,platform6_data_clean]

In [737]:
model=RandomForestClassifier()
kfold_validation=KFold(5)

In [738]:
for clean_data in clean_data_list:
  x = clean_data.iloc[:,:6]
  y = clean_data.iloc[:,-1]
  results=cross_val_score(model,x,y,cv=kfold_validation)
  name = namestr(clean_data,globals())
  print(f'Result of {name[0]}\n')
  print(results)
  print(np.mean(results),'\n')
  print('************************')
  print('\n')

Result of clean_data

[0.49640288 0.52517986 0.44604317 0.54676259 0.52517986]
0.5079136690647481 

************************


Result of clean_data

[0.5        0.39130435 0.55555556 0.53333333 0.62222222]
0.5204830917874397 

************************


Result of clean_data

[0.4375     0.48387097 0.41935484 0.48387097 0.48387097]
0.4616935483870968 

************************


Result of clean_data

[0.5        0.75       0.66666667 0.33333333 0.33333333]
0.5166666666666667 

************************


Result of clean_data

[0.51626016 0.48979592 0.52244898 0.50612245 0.53061224]
0.5130479508876722 

************************




According the results we got above using the 5-fold validation randomforset classifier the maximum accuracy we got is 0.75 from all the datasets. Following this we are can deduce that either the data used is not enought or the model is not suitable as the accuracy is less than 80%